# Download de l'ensemble des bases de données 

## Purpose de ce notebook : 

- Download de l'ensemble des bases de données nécessaire au travail de visualisation dans le cadre de notre projet 
- Cleaning de la base de données afin de pouvoir faire des manipulations dans le cadre de notre projet (drop de colonne, filtrage, etc...)

### Import des bibliothèques nécessaires 

In [10]:
!pip install chardet 
!pip install nbformat
import requests
import zipfile
import os
import pandas as pd
import geopandas as gpd

In [5]:

# Dictionnaire des URLs par année
urls = {
    2022: 'https://www.data.gouv.fr/fr/datasets/r/36314eff-2712-4d6a-b091-7752a5b318bb',
    2021: 'https://www.data.gouv.fr/fr/datasets/r/e6cd3b9f-38e2-47ca-aac6-5acfb9567e01',
    2023: 'https://www.data.gouv.fr/fr/datasets/r/e8639487-199c-4e30-8998-1b517148fe29',
}

for year, url_zip in urls.items():
    print(f"\nTraitement de l'année {year}...")

    # Chemin pour stocker temporairement le fichier téléchargé
    file_path = f"data_{year}.zip"

    # Télécharger le fichier
    response = requests.get(url_zip)
    if response.status_code != 200:
        print(f"Erreur lors du téléchargement pour l'année {year}: {response.status_code}")
        continue

    # Enregistrer le fichier localement
    with open(file_path, "wb") as f:
        f.write(response.content)

    # Vérifier si le fichier est un ZIP
    if zipfile.is_zipfile(file_path):
        print(f"Extraction du fichier ZIP pour l'année {year}")
        extract_path = f"data_extracted_{year}"
        if not os.path.exists(extract_path):
            os.makedirs(extract_path)
        with zipfile.ZipFile(file_path, "r") as zip_ref:
            zip_ref.extractall(extract_path)

        # Afficher tous les fichiers extraits pour diagnostic
        print(f"Fichiers extraits pour {year} :")
        extracted_files = []
        for root, _, files in os.walk(extract_path):
            for file in files:
                extracted_files.append(os.path.join(root, file))
                print(f"- {file}")

        # Lire les fichiers extraits si ce sont des CSV
        for file in extracted_files:
            if file.endswith(".csv"):
                gdf = pd.read_csv(file)
                globals()[f"gdf{year}"] = gdf.drop(columns='geometry', errors='ignore')
                print(f"Aperçu des données pour {year} :")
                print(globals()[f"gdf{year}"].head())
    else:
        print(f"Le fichier pour {year} n'est pas un ZIP.")



Traitement de l'année 2022...
Extraction du fichier ZIP pour l'année 2022
Fichiers extraits pour 2022 :
- rpg-bio-2022-national.gpkg
- ._rpg-bio-2022-national.gpkg

Traitement de l'année 2021...
Extraction du fichier ZIP pour l'année 2021
Fichiers extraits pour 2021 :
- rpg-bio-2021-national.gpkg

Traitement de l'année 2023...
Extraction du fichier ZIP pour l'année 2023
Fichiers extraits pour 2023 :
- rpg-bio-2023-national.gpkg
- rpg-bio-2023-national.prj
- rpg-bio-2023-national.cpg
- rpg-bio-2023-national.shx
- rpg-bio-2023-national.dbf
- rpg-bio-2023-national.shp


### Stockage des données dans des variables 

In [12]:
# Chemins vers les fichiers extraits
file_2022 = "data_extracted_2022/rpg-bio-2022-national.gpkg"
file_2021 = "data_extracted_2021/rpg-bio-2021-national.gpkg"
file_2023 = "data_extracted_2023/rpg-bio-2023-national/rpg-bio-2023-national.gpkg"

# Charger les fichiers dans des variables distinctes
gdf2022 = gpd.read_file(file_2022)
gdf2021 = gpd.read_file(file_2021)
gdf2023 = gpd.read_file(file_2023)

# Afficher un aperçu des données
print("Aperçu des données pour 2022 :")
print(gdf2022.head())

print("\nAperçu des données pour 2021 :")
print(gdf2021.head())

print("\nAperçu des données pour 2023 :")
print(gdf2023.head())

Aperçu des données pour 2022 :
       gid  millesime code_culture  \
0  4420687       2022          PPH   
1      333       2022          PTR   
2      337       2022          BTH   
3      464       2022          BTH   
4      480       2022          BTH   

                                         lbl_culture           grp_culture  \
0  Prairie permanente - herbe prédominante (resso...  Prairies permanentes   
1         Autre prairie temporaire de 5 ans ou moins  Prairies temporaires   
2                                 Blé tendre d’hiver            Blé tendre   
3                                 Blé tendre d’hiver            Blé tendre   
4                                 Blé tendre d’hiver            Blé tendre   

   surface_ha code_commune_insee             lbl_commune    code_epci  \
0        0.38              38513                Treffort  200030658.0   
1        2.13              01188                  Illiat  200070118.0   
2        6.10              01136  Cruzilles-lès-Mépi

In [19]:
gdf2021.columns

Index(['gid', 'millesime', 'code_culture', 'lbl_culture', 'grp_culture',
       'surface_ha', 'code_commune_insee', 'lbl_commune', 'code_epci',
       'code_departement', 'code_region', 'zone_geo', 'geometry'],
      dtype='object')

### Automatisation du cleaning 

In [18]:
# Dictionnaire des mappings pour les régions
region_mapping = {
    '11': 'Île-de-France',
    '24': 'Centre-Val de Loire',
    '27': 'Bourgogne-Franche-Comté',
    '28': 'Normandie',
    '32': 'Hauts-de-France',
    '44': 'Grand Est',
    '52': 'Pays de la Loire',
    '53': 'Bretagne',
    '75': 'Nouvelle-Aquitaine',
    '76': 'Occitanie',
    '84': 'Auvergne-Rhône-Alpes',
    '93': 'Provence-Alpes-Côte d\'Azur',
    '94': 'Corse'
}

# Liste des codes régionaux pour la métropole
metropolitan_codes = [11, 24, 27, 28, 32, 44, 52, 53, 75, 76, 84, 93, 94]

# Fonction de nettoyage pour une base de données donnée
def clean_and_process_data(gdf, year):
    # Étape 1 : Supprimer la colonne geometry si elle existe
    gdf = gdf.drop(columns=['geometry'], errors='ignore')
    
    # Étape 2 : Compter le nombre de parcelles biologiques par département
    parcelles_par_departement = gdf['code_depar'].value_counts().reset_index()
    parcelles_par_departement.columns = ['code_departement', 'nombre_parcelles']
    
    # Étape 3 : Agréger la surface totale des parcelles biologiques par département
    surface_par_departement = gdf.groupby('code_departement')['surface_ha'].sum().reset_index()
    surface_par_departement.columns = ['code_departement', 'surface_totale_ha']
    
    # Fusionner les deux DataFrames
    resultat = parcelles_par_departement.merge(surface_par_departement, on='code_departement')
    resultat = resultat.sort_values(by='surface_totale_ha', ascending=False)
    
    # Étape 4 : Filtrer pour les régions métropolitaines
    gdf = gdf[gdf['code_regio'].astype(str).isin(region_mapping.keys())]
    
    # Ajouter une colonne nom_region à partir des codes
    gdf['nom_region'] = gdf['code_region'].astype(str).map(region_mapping)
    
    # Filtrer pour les codes régionaux métropolitains
    gdf_metropole = gdf[gdf['code_region'].isin(metropolitan_codes)]
    
    # Calculer le total des parcelles bio en métropole
    total_metropole = len(gdf_metropole)
    print(f"Le nombre total de parcelles bio en métropole pour {year} est :", total_metropole)
    
    # Retourner les résultats et le DataFrame nettoyé
    return gdf, resultat

# Appliquer le nettoyage pour chaque année
for year in urls.keys():
    print(f"\nTraitement des données pour l'année {year}...")
    try:
        # Charger le fichier correspondant à l'année
        gdf = globals().get(f"gdf{year}")
        if gdf is not None:
            # Appliquer la fonction de nettoyage
            gdf_cleaned, resultat = clean_and_process_data(gdf, year)
            
            # Afficher les premiers résultats
            print("Aperçu des données nettoyées :")
            print(resultat.head())
        else:
            print(f"Aucune donnée trouvée pour l'année {year}.")
    except Exception as e:
        print(f"Erreur lors du traitement des données pour {year} : {e}")



Traitement des données pour l'année 2022...
Erreur lors du traitement des données pour 2022 : 'code_depar'

Traitement des données pour l'année 2021...
Erreur lors du traitement des données pour 2021 : 'code_depar'

Traitement des données pour l'année 2023...
Erreur lors du traitement des données pour 2023 : 'code_depar'
